# Exploratory Data Analysis and Hypothesis Testing

### Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA

#import missingno as msno
import warnings

# Significance tests
from scipy.stats import mannwhitneyu

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
#import xgboost as xgb
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")
#pd.set_option('display.max_rows')
#pd.reset_option(“max_columns”)

# Imports function from GitHub
!curl -Os https://raw.githubusercontent.com/jgeorge313/DS_GA_1001_Capstone/main/Contributors/Helper_Functions.py
%run Helper_Functions.py

In [4]:
ds_dataset = pd.read_csv('https://raw.githubusercontent.com/jgeorge313/DS_GA_1001_Capstone/main/Levels_Fyi_Salary_Data.csv')
F1000_dataset = pd.read_csv('https://raw.githubusercontent.com/jgeorge313/DS_GA_1001_Capstone/main/Fortune_1000.csv')

ds_dataset['company'] = ds_dataset['company'].str.lower()
F1000_dataset['company'] = F1000_dataset['company'].str.lower()
raw_df = pd.merge(ds_dataset, F1000_dataset, how='left', on='company')
raw_df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,state,newcomer,ceo_founder,ceo_woman,profitable,prev_rank,CEO,Website,Ticker,Market Cap
0,6/7/2017 11:33:27,oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000.0,...,CA,no,no,yes,yes,81.0,Safra A. Catz,https://www.oracle.com,ORCL,170158
1,6/10/2017 17:11:29,ebay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0.0,...,CA,no,no,no,yes,295.0,Jamie Iannone,https://www.ebay.com,EBAY,23930.2
2,6/11/2017 14:53:57,amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000.0,...,WA,no,yes,no,yes,5.0,Jeffrey P. Bezos,https://www.amazon.com,AMZN,1637405
3,6/17/2017 0:23:14,apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,...,CA,no,no,no,yes,3.0,Timothy D. Cook,https://www.apple.com,AAPL,2221176
4,6/20/2017 10:58:51,microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0.0,...,WA,no,no,no,yes,26.0,Satya Nadella,https://www.microsoft.com,MSFT,1696122


In [5]:
raw_df = match_industry(raw_df) # Fills in sector 

#Rounds the number of years of experience (reduces number of categories)
raw_df.loc[:,'yearsofexperience'] = np.round(raw_df.loc[:,'yearsofexperience'])
raw_df.loc[:,'yearsatcompany'] = np.round(raw_df.loc[:,'yearsatcompany'])

#Drop users that had a higher value for yearsatcompany than yearsofexperience
raw_df.drop(raw_df[(raw_df.loc[:,'yearsatcompany'] - raw_df.loc[:,'yearsofexperience']) > 0].index.to_list(), inplace=True)

#Fill in missing values for dmaid
raw_df['dmaid'].fillna(raw_df['dmaid'].median(), inplace=True)

#Changes the category names of education to fit into plot
raw_df['Education'].replace({"Master's Degree": "Master's", "Bachelor's Degree": "Bachelor's"}, inplace=True)

#Add region, faang, year columns and cleans the tag and sector columns
raw_df = extract_state(raw_df)
raw_df = extract_year(raw_df)
raw_df = extract_tag(raw_df)
raw_df = faang(raw_df)
raw_df = match_industry(raw_df)

# Deletes wrong input in gender
raw_df.drop(raw_df[raw_df['gender'] == 'Title: Senior Software Engineer'].index, inplace=True)

In [6]:
df = raw_df[['company', 'title', 'totalyearlycompensation', 'location', 'yearsofexperience', 'yearsatcompany', 
             'gender', 'Education', 'Race', 'sector', 'region', 'faang', 'dmaid', 'tag', 'year']]

## MissingNo plots

In [7]:
msno.bar(df)

NameError: name 'msno' is not defined

## Distributions of categorical features versus quantitative features

In [ ]:
categorical_features = ['gender', 'Race', 'Education', 'faang']

plot_features(df, categorical_features, is_categorical=True)

In [ ]:
continuous_features = ['yearsofexperience', 'yearsatcompany', 'totalyearlycompensation', 'dmaid']

plot_features(df, continuous_features, is_categorical=False)

## Hypothesis Testing

#### Hypothesis 1 (When controlling for experience, do people with high vs. low experience years at current company make more?)

$H_0$: People that have worked longer in a company earn equal salaries than people that have not worked as long in that same company.

$H_1$: People that have worked longer in a company earn significantly different salaries than people that have not worked as long in that same company.<br>

**Controlling for 10 to 15 years of experience*

In [9]:
test_input1 = hypothesis_data1(df, 'yearsofexperience', 'yearsatcompany', 'totalyearlycompensation', [5, 10, 15])

In [10]:
for key in test_input1.keys():
    test = mannwhitneyu(test_input1[key][0], test_input1[key][1], alternative='two-sided') #Runs a Mann Whitney U-test
    if test.pvalue < 0.05:
        print('{} years of experience: \nWe reject the Null Hypothesis (p-value = {})\n'.format(key, test.pvalue))
    else:
        print('{} years of experience: \nWe fail to reject the Null Hypothesis (p-value = {})\n'.format(key, test.pvalue))

0-5 years of experience: 
We reject the Null Hypothesis (p-value = 2.350051517848951e-13)

5-10 years of experience: 
We reject the Null Hypothesis (p-value = 2.612796616631838e-24)

10-15 years of experience: 
We reject the Null Hypothesis (p-value = 3.13707728710554e-19)

15-69.0 years of experience: 
We reject the Null Hypothesis (p-value = 3.684134271046804e-21)



#### Hypothesis 2 (When controlling for experience, do people that work at FAANG earn higher salaries than people that don't?)

$H_0$: People that work at FAANG earn the same salary than other non-FAANG tech workers. <br>
$H_1$: People that work at FAANG earn a significantly different salary than other non-FAANG tech workers.<br>

**Controlling for 10 to 15 years of experience*

In [11]:
for key in test_input1.keys():
    print('{} years of experience: \nlow experience: ${}, high experience: ${}\n'.format(key, int(test_input1[key][0].median()), int(test_input1[key][1].median())))

0-5 years of experience: 
low experience: $167000, high experience: $172000

5-10 years of experience: 
low experience: $225000, high experience: $210000

10-15 years of experience: 
low experience: $270000, high experience: $250000

15-69.0 years of experience: 
low experience: $306000, high experience: $280000



In [12]:
test_input2 = hypothesis_data2(df, 'yearsofexperience', 'sector', 'company', 'totalyearlycompensation', [5, 10, 15])

In [13]:
for key in test_input2.keys():
    test = mannwhitneyu(test_input2[key][0], test_input2[key][1], alternative='two-sided') #Runs a Mann Whitney U-test
    if test.pvalue < 0.05:
        print('{} years of experience: \nWe reject the Null Hypothesis (p-value = {})\n'.format(key, test.pvalue))
    else:
        print('{} years of experience: \nWe fail to reject the Null Hypothesis (p-value = {})\n'.format(key, test.pvalue))

0-5 years of experience: 
We reject the Null Hypothesis (p-value = 2.5704808550357074e-220)

5-10 years of experience: 
We reject the Null Hypothesis (p-value = 7.500084048882348e-151)

10-15 years of experience: 
We reject the Null Hypothesis (p-value = 1.681324512146866e-116)

15-69.0 years of experience: 
We reject the Null Hypothesis (p-value = 1.3318807798551037e-107)



In [14]:
for key in test_input2.keys():
    print('{} years of experience: \nfaang: ${}, non-faang: ${}\n'.format(key, int(test_input2[key][0].median()), int(test_input2[key][1].median())))

0-5 years of experience: 
faang: $200000, non-faang: $173000

5-10 years of experience: 
faang: $261000, non-faang: $220000

10-15 years of experience: 
faang: $320000, non-faang: $260000

15-69.0 years of experience: 
faang: $375000, non-faang: $290000



In [ ]:
#### Hypothesis 3 (When controlling for experience, do men and women earn significantly different pay?)